In [ ]:
!pip install huggingface_hub
!pip install chromadb
!pip install langchain
!pip install pypdf
!pip install sentence-transformers

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
import sys

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
api_key = os.getenv("MY_API_KEY")


model_name = "mistralai/Mistral-7B-v0.1"
model = HuggingFaceHub(
    huggingfacehub_api_token="MY_API_KEY",
    repo_id=model_name,
    model_kwargs={
        "temperature": 0.2,
        "max_new_tokens": 50
    }
)

In [ ]:
!git clone https://github.com/boratsbias/rag-101
%cd rag-101

doc_loader = PyPDFLoader('sample_doc.pdf')
docs = doc_loader.load()

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = splitter.split_documents(docs)

fatal: destination path 'rag-101' already exists and is not an empty directory.
/content/rag-101/rag-101


In [ ]:
query_embedder = HuggingFaceEmbeddings()

vector_db = Chroma.from_documents(chunks, query_embedder)

top_k = 2
retriever = vector_db.as_retriever(search_kwargs={'k': top_k})

<ipython-input-10-bdcef19b9124>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  query_embedder = HuggingFaceEmbeddings()


In [ ]:
chain = ConversationalRetrievalChain.from_llm(model, retriever=retriever, return_source_documents=False)

In [ ]:
chats = []
while True:
    prompt = input('Prompt: ')
    if prompt == 'q':
        sys.exit()
    response = chain({'question': prompt, 'chat_history': chats})
    answer = response['answer']
    helpful_answer = answer.find('Helpful Answer')
    if helpful_answer != -1:
        print(answer[helpful_answer:] + '\n')
    chats.append((prompt, answer))

Prompt: who won the awards?


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


HfHubHTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://router.huggingface.co/hf-inference/models/mistralai/Mistral-7B-v0.1